In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [2]:
# imporing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# set the display option to show up t0 100 rows
pd.set_option('display.max_rows',100)

In [4]:
# importing the dataset
df = pd.read_csv('/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv')

print(df[:10])

  day_id city_id  temperature_celsius           condition_text  wind_kph  \
0  D0001    C001                 27.0                      NaN       6.1   
1  D0002    C001                 22.0                      NaN       6.1   
2  D0003    C001                 20.0  Light Rain with Thunder       3.6   
3  D0004    C001                 17.0          Clear and Sunny       6.1   
4  D0005    C001                 18.0                      NaN       3.6   
5  D0006    C001                 20.0                      NaN       3.6   
6  D0007    C001                 21.0            Partly Cloudy       4.0   
7  D0008    C001                 21.0                      NaN      20.2   
8  D0009    C001                 23.0                      NaN       3.6   
9  D0010    C001                 24.0                      NaN      15.1   

   wind_degree  pressure_mb  precip_mm  humidity  cloud  feels_like_celsius  \
0          210       1006.0        0.0        54     75                28.0   
1    

In [5]:
# dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   day_id                    2893 non-null   object 
 1   city_id                   2893 non-null   object 
 2   temperature_celsius       2893 non-null   float64
 3   condition_text            479 non-null    object 
 4   wind_kph                  2893 non-null   float64
 5   wind_degree               2893 non-null   int64  
 6   pressure_mb               2893 non-null   float64
 7   precip_mm                 2893 non-null   float64
 8   humidity                  2893 non-null   int64  
 9   cloud                     2893 non-null   int64  
 10  feels_like_celsius        2893 non-null   float64
 11  visibility_km             2893 non-null   float64
 12  uv_index                  2893 non-null   float64
 13  gust_kph                  2893 non-null   float64
 14  air_qual

In [6]:
# analyzing the dataset
def df_analyze(dataframe):
    df = pd.DataFrame()
    cl=[]; u=[]; s=[]; nans=[]
    
    for col in dataframe.columns:
        cl.append(col); u.append(dataframe[col].unique()); s.append(dataframe[col].unique().size); nans.append(dataframe[col].isnull().sum()) 
        
    df['Columns']=cl; df['Uniques']=u; df['Cardinality']=s; df['NaNs']=nans;

    return df

df_info = df_analyze(df)
df_info

,Columns,Uniques,Cardinality,NaNs
0,day_id,"[D0001, D0002, D0003, D0004, D0005, D0006, D00...",2893,0
1,city_id,"[C001, C002, C003, C004, C005, C006, C007, C00...",112,0
2,temperature_celsius,"[27.0, 22.0, 20.0, 17.0, 18.0, 21.0, 23.0, 24....",191,0
3,condition_text,"[nan, Light Rain with Thunder, Clear and Sunny...",10,2414
4,wind_kph,"[6.1, 3.6, 4.0, 20.2, 15.1, 6.8, 13.0, 16.9, 1...",84,0
5,wind_degree,"[210, 170, 10, 150, 92, 96, 310, 330, 63, 320,...",281,0
6,pressure_mb,"[1006.0, 1011.0, 1018.0, 1019.0, 1015.0, 1013....",49,0
7,precip_mm,"[0.0, 4.5, 0.32, 6.4, 0.14, 0.1, 0.3, 0.2, 0.2...",122,0
8,humidity,"[54, 73, 100, 88, 94, 53, 47, 57, 83, 45, 78, ...",94,0
9,cloud,"[75, 0, 50, 100, 25, 62, 37, 70, 10, 28, 80, 2...",87,0


In [7]:
# converting the sunset and sunrise columns into datetime format
for state in ['sunrise', 'sunset']:
    df[state] = pd.to_datetime(df[state])
    
    df[f'{state}_hour'] = df[state].dt.hour
    df[f'{state}_min'] = df[state].dt.minute
    
df = df.drop(['sunrise', 'sunset'], axis=1)
df[:10]

/tmp/ipykernel_95/1513255587.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[state] = pd.to_datetime(df[state])
/tmp/ipykernel_95/1513255587.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[state] = pd.to_datetime(df[state])


,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_hour,sunrise_min,sunset_hour,sunset_min
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,6,4,19,19
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,6,5,19,18
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,6,5,19,18
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,6,6,19,16
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,6,7,19,15
5,D0006,C001,20.0,NaN,3.6,96,1019.0,0.0,88,0,20.0,10.0,1.0,11.2,1,6,7,19,13
6,D0007,C001,21.0,Partly Cloudy,4.0,310,1015.0,0.0,100,50,21.0,10.0,1.0,15.1,2,6,8,19,11
7,D0008,C001,21.0,NaN,20.2,330,1011.0,0.0,53,75,21.0,10.0,1.0,17.3,1,6,9,19,10
8,D0009,C001,23.0,NaN,3.6,63,1013.0,0.0,47,75,24.8,10.0,1.0,20.5,1,6,10,19,8
9,D0010,C001,24.0,NaN,15.1,330,1014.0,0.0,54,75,25.3,10.0,1.0,9.7,1,6,11,19,6


In [8]:
# encoding categorical variables
df['city_id'] = df['city_id'].astype('category').cat.codes

In [9]:
# encoding the day ids
df['day_id'] = pd.factorize(df['day_id'])[0] + 1
df

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise_hour,sunrise_min,sunset_hour,sunset_min
0,1,0,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,6,4,19,19
1,2,0,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,6,5,19,18
2,3,0,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,6,5,19,18
3,4,0,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,6,6,19,16
4,5,0,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,6,7,19,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,2889,111,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,10.0,1.0,22.2,1,5,49,17,51
2889,2890,111,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,10.0,1.0,22.2,1,5,49,17,52
2890,2891,111,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,10.0,1.0,21.3,2,5,48,17,52
2891,2892,111,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,10.0,1.0,24.9,2,5,47,17,52


In [10]:
# analyzing the dataset
df_info = df_analyze(df)
df_info

,Columns,Uniques,Cardinality,NaNs
0,day_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2893,0
1,city_id,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",112,0
2,temperature_celsius,"[27.0, 22.0, 20.0, 17.0, 18.0, 21.0, 23.0, 24....",191,0
3,condition_text,"[nan, Light Rain with Thunder, Clear and Sunny...",10,2414
4,wind_kph,"[6.1, 3.6, 4.0, 20.2, 15.1, 6.8, 13.0, 16.9, 1...",84,0
5,wind_degree,"[210, 170, 10, 150, 92, 96, 310, 330, 63, 320,...",281,0
6,pressure_mb,"[1006.0, 1011.0, 1018.0, 1019.0, 1015.0, 1013....",49,0
7,precip_mm,"[0.0, 4.5, 0.32, 6.4, 0.14, 0.1, 0.3, 0.2, 0.2...",122,0
8,humidity,"[54, 73, 100, 88, 94, 53, 47, 57, 83, 45, 78, ...",94,0
9,cloud,"[75, 0, 50, 100, 25, 62, 37, 70, 10, 28, 80, 2...",87,0


In [11]:
# importing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [12]:
# create training and validation sets
train_df = df[df['condition_text'].notnull()].copy()
val_df = df[df['condition_text'].isnull()].copy()

# drop the 'condition_text' column from the training set
X_dum = train_df.drop('condition_text', axis=1)
y_dum = train_df['condition_text']

# drop the 'condition_text' column from the validation set
X_val = val_df.drop('condition_text', axis=1)
y_val = val_df['condition_text']

# check the lengths match
assert len(df) == len(train_df) + len(val_df)

# check the lengths match for the target column
assert len(df[df['condition_text'].notnull()]) == len(y_dum)
assert len(df[df['condition_text'].isnull()]) == len(y_val)

# check the lengths match for the features
assert len(X_dum) == len(train_df)
assert len(X_val) == len(val_df)

# display the first few rows of the training set
print("Training set:")
print(len(train_df))
print(train_df[:5])

# display the first few rows of the validation set
print("\nValidation set:")
print(len(val_df))
print(val_df[:5])

Training set:
479
    day_id  city_id  temperature_celsius           condition_text  wind_kph  \
2        3        0                 20.0  Light Rain with Thunder       3.6   
3        4        0                 17.0          Clear and Sunny       6.1   
6        7        0                 21.0            Partly Cloudy       4.0   
18      19        0                 19.0          Clear and Sunny       3.6   
27      28        1                 19.0            Partly Cloudy       3.6   

    wind_degree  pressure_mb  precip_mm  humidity  cloud  feels_like_celsius  \
2            10       1011.0        4.5       100     75                20.0   
3           150       1018.0        0.0        88      0                17.0   
6           310       1015.0        0.0       100     50                21.0   
18           64       1017.0        0.0        88      0                19.0   
27           83       1010.0        0.0        73     25                19.0   

    visibility_km  uv_inde

In [13]:
# analyzing the dataset
df_info = df_analyze(train_df)
df_info

,Columns,Uniques,Cardinality,NaNs
0,day_id,"[3, 4, 7, 19, 28, 35, 42, 79, 81, 87, 99, 103,...",479,0
1,city_id,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",110,0
2,temperature_celsius,"[20.0, 17.0, 21.0, 19.0, 25.0, 29.0, 27.0, 11....",81,0
3,condition_text,"[Light Rain with Thunder, Clear and Sunny, Par...",9,0
4,wind_kph,"[3.6, 6.1, 4.0, 15.1, 9.0, 24.1, 28.1, 20.2, 1...",56,0
5,wind_degree,"[10, 150, 310, 64, 83, 70, 141, 90, 100, 80, 1...",134,0
6,pressure_mb,"[1011.0, 1018.0, 1015.0, 1017.0, 1010.0, 1016....",33,0
7,precip_mm,"[4.5, 0.0, 0.3, 0.1, 0.24, 0.02, 0.2, 0.04, 0....",55,0
8,humidity,"[100, 88, 73, 83, 79, 70, 75, 82, 94, 77, 51, ...",70,0
9,cloud,"[75, 0, 50, 25, 62, 100, 70, 80, 53, 48, 59, 8...",51,0


In [14]:
# build the classification model
clf = RandomForestClassifier()

In [15]:
# data split
X_train, X_test, y_train, y_test = train_test_split(X_dum, y_dum, test_size=0.2, random_state=42)

In [16]:
# getting the shape of the training data
X_train.shape, y_train.shape

((383, 18), (383,))

In [17]:
# getting the shape of the test data
X_test.shape, y_test.shape

((96, 18), (96,))

In [18]:
# rebuilding the model
clf.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
# predicting the target on the test set
y_pred = clf.predict(X_test)

In [20]:
# create a DataFrame to display y_test and y_pred side by side
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# display the results side by side
print(results_df[:30])

                       Actual            Predicted
624       Cloudy and Overcast  Cloudy and Overcast
1790            Partly Cloudy        Partly Cloudy
345       Light Precipitation  Light Precipitation
237       Light Precipitation  Light Precipitation
2715            Partly Cloudy        Partly Cloudy
2747            Partly Cloudy        Partly Cloudy
2519   Moderate to Heavy Rain        Partly Cloudy
524       Cloudy and Overcast  Cloudy and Overcast
1515            Partly Cloudy        Partly Cloudy
86        Cloudy and Overcast        Partly Cloudy
1340          Clear and Sunny          Mist or Fog
2779            Partly Cloudy        Partly Cloudy
676              Rain Showers         Rain Showers
1547      Cloudy and Overcast        Partly Cloudy
1598            Partly Cloudy        Partly Cloudy
553               Mist or Fog          Mist or Fog
1536            Partly Cloudy        Partly Cloudy
2447            Partly Cloudy        Partly Cloudy
1855            Partly Cloudy  

In [21]:
# predicting the missing classes
y_val = clf.predict(X_val)
y_val[:10]

array(['Partly Cloudy', 'Partly Cloudy', 'Clear and Sunny',
       'Clear and Sunny', 'Partly Cloudy', 'Partly Cloudy',
       'Partly Cloudy', 'Clear and Sunny', 'Clear and Sunny',
       'Clear and Sunny'], dtype=object)

In [22]:
# getting the accuracy of the model
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7083333333333334


In [23]:
# concatenate y_val and X_val
y_val_df = pd.DataFrame({'condition_text': y_val})
concatenated_val_df = pd.concat([X_val.reset_index(drop=True), y_val_df.reset_index(drop=True)], axis=1)

# display the concatenated dataset
print(concatenated_val_df.head())

   day_id  city_id  temperature_celsius  wind_kph  wind_degree  pressure_mb  \
0       1        0                 27.0       6.1          210       1006.0   
1       2        0                 22.0       6.1          170       1006.0   
2       5        0                 18.0       3.6           92       1019.0   
3       6        0                 20.0       3.6           96       1019.0   
4       8        0                 21.0      20.2          330       1011.0   

   precip_mm  humidity  cloud  feels_like_celsius  visibility_km  uv_index  \
0        0.0        54     75                28.0           10.0       6.0   
1        0.0        73     75                24.5           10.0       1.0   
2        0.0        94      0                18.0           10.0       1.0   
3        0.0        88      0                20.0           10.0       1.0   
4        0.0        53     75                21.0           10.0       1.0   

   gust_kph  air_quality_us-epa-index  sunrise_hour  sun

In [24]:
# concatenate train_df and concatenated_val_df
final_df = pd.concat([train_df, concatenated_val_df], ignore_index=True)

# sort by day id
final_df = final_df.sort_values(by='day_id')

# add a new column with day IDs formatted as text
final_df['day_id'] = 'D' + final_df['day_id'].astype(str).str.zfill(4)

# display the final concatenated dataset
print(final_df.shape)

(2893, 19)


In [25]:
# analyzing the dataset
df_info = df_analyze(final_df)
df_info

,Columns,Uniques,Cardinality,NaNs
0,day_id,"[D0001, D0002, D0003, D0004, D0005, D0006, D00...",2893,0
1,city_id,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",112,0
2,temperature_celsius,"[27.0, 22.0, 20.0, 17.0, 18.0, 21.0, 23.0, 24....",191,0
3,condition_text,"[Partly Cloudy, Light Rain with Thunder, Clear...",9,0
4,wind_kph,"[6.1, 3.6, 4.0, 20.2, 15.1, 6.8, 13.0, 16.9, 1...",84,0
5,wind_degree,"[210, 170, 10, 150, 92, 96, 310, 330, 63, 320,...",281,0
6,pressure_mb,"[1006.0, 1011.0, 1018.0, 1019.0, 1015.0, 1013....",49,0
7,precip_mm,"[0.0, 4.5, 0.32, 6.4, 0.14, 0.1, 0.3, 0.2, 0.2...",122,0
8,humidity,"[54, 73, 100, 88, 94, 53, 47, 57, 83, 45, 78, ...",94,0
9,cloud,"[75, 0, 50, 100, 25, 62, 37, 70, 10, 28, 80, 2...",87,0


In [26]:
# select the neccessary columns
submission_key_final = final_df[['day_id', 'condition_text']]
print(submission_key_final[:20])

# save to a new CSV file
submission_key_final.to_csv('submission_condition_class.csv', index=False)

# display the saved file for download
from IPython.display import FileLink

# provide a link to download the file
FileLink('submission_condition_class.csv')

    day_id           condition_text
479  D0001            Partly Cloudy
480  D0002            Partly Cloudy
0    D0003  Light Rain with Thunder
1    D0004          Clear and Sunny
481  D0005          Clear and Sunny
482  D0006          Clear and Sunny
2    D0007            Partly Cloudy
483  D0008            Partly Cloudy
484  D0009            Partly Cloudy
485  D0010            Partly Cloudy
486  D0011          Clear and Sunny
487  D0012          Clear and Sunny
488  D0013          Clear and Sunny
489  D0014          Clear and Sunny
490  D0015          Clear and Sunny
491  D0016          Clear and Sunny
492  D0017            Partly Cloudy
493  D0018          Clear and Sunny
3    D0019          Clear and Sunny
494  D0020      Cloudy and Overcast


/kaggle/working/submission_condition_class.csv